In [38]:
# import modules
import os, sys
import requests
import pandas as pd
from tqdm import tqdm
tqdm.pandas() # for progress_apply
from fuzzywuzzy import fuzz
import numpy as np


In [50]:
company = pd.read_pickle('processed_companyData.pkl') #company market data
lda2013_Q1 = pd.read_csv('filings_2013_Q1.csv') #lobbying data of 2023 Q1


/var/folders/ks/l1jjkq0j6s34v3pvsq4_bxjr0000gn/T/ipykernel_6276/3911776069.py:2: DtypeWarning: Columns (20,117) have mixed types. Specify dtype option on import or set low_memory=False.
  lda2013_Q1 = pd.read_csv('filings_2013_Q1.csv') #lobbying data of 2023 Q1


In [51]:
# Move c_id and c_name to the first 2 columns

columns_to_move = ['c_id', 'c_name']  # Example columns to move
new_positions = [0, 1]  # New positions for the columns to move

# Get the current order of columns
current_columns = lda2013_Q1.columns.tolist()

# Remove columns to move from the current order
for col in columns_to_move:
    current_columns.remove(col)

# Insert columns to move at their new positions
for col, pos in zip(columns_to_move, new_positions):
    current_columns.insert(pos, col)

# Reassign the columns in the DataFrame with the new order
lda2013_Q1 = lda2013_Q1[current_columns]

In [52]:
pd.set_option('display.max_columns', 148)
lda2013_Q1.head(2)

,c_id,c_name,url,filing_uuid,filing_type,filing_type_display,filing_year,filing_period,filing_period_display,filing_document_url,filing_document_content_type,income,expenses,expenses_method,expenses_method_display,posted_by_name,dt_posted,termination_date,registrant_country,registrant_ppb_country,registrant_address_1,registrant_address_2,registrant_different_address,registrant_city,registrant_state,registrant_zip,registrant,client,lobbying_activities,conviction_disclosures,foreign_entities,affiliated_organizations,issue_codes,i_RES,i_HCR,i_AER,i_FIR,i_ECN,i_REL,i_ANI,i_PHA,i_UTI,i_ACC,i_TRD,i_TOB,i_FIN,i_RET,i_SMB,i_FUE,i_FOR,i_INS,i_CPT,i_TOU,i_HOM,i_BAN,i_MMM,i_TEC,i_MIA,i_EDU,i_TAX,i_SPO,i_IMM,i_ENG,i_ROD,i_TOR,i_NAT,i_TRA,i_BEV,i_AUT,i_ART,i_ALC,i_RRR,i_SCI,i_CPI,i_HOU,i_DEF,i_GAM,i_DOC,i_WAS,i_CDT,i_MED,i_URB,i_LAW,i_ENV,i_ADV,i_CON,i_IND,i_WEL,i_APP,i_COM,i_INT,i_UNM,i_VET,i_MAN,i_LBR,i_GOV,i_DIS,i_MAR,i_MON,i_AGR,i_FOO,i_CSP,i_BNK,i_AVI,i_FAM,i_TAR,i_POS,i_BUD,i_CHM,i_CIV,i_CAW,i_TRU,i_NUMBER,i_EMPTY,c_url,c_client_id,c_general_description,c_client_government_entity,c_client_self_select,c_state,c_state_display,c_country,c_country_display,c_ppb_state,c_ppb_state_display,c_ppb_country,c_ppb_country_display,c_effective_date,r_id,r_url,r_house_registrant_id,r_name,r_description,r_address_1,r_address_2,r_address_3,r_address_4,r_city,r_state,r_state_display,r_zip,r_country,r_country_display,r_ppb_country,r_ppb_country_display,r_contact_name,r_contact_telephone,r_dt_updated
0,193537,THE COLLEGE OF WILLIAM & MARY,https://lda.senate.gov/api/v1/filings/ac81d9c2...,ac81d9c2-a5d6-4051-ad52-6fb98db8fdfb,Q1,1st Quarter - Report,2013,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/a...,text/html,NaN,25000.00,a,Method A - Reporting amounts using LDA definit...,"Michael J. Connolly, Director of W&M Federal R...",2013-01-18T15:47:41.127000-05:00,NaN,United States of America,NaN,228 Jamestown Road,NaN,False,Williamsburg,VA,23185.0,"{'id': 400614862, 'url': 'https://lda.senate.g...","{'id': 193537, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'TRA', 'general_issue_...",[],[],[],"['TRA', 'ENV', 'VET']",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,https://lda.senate.gov/api/v1/clients/193537/,12,NaN,True,True,VA,Virginia,US,United States of America,VA,Virginia,US,United States of America,2010-07-01,400614862,https://lda.senate.gov/api/v1/registrants/4006...,400614862.0,THE COLLEGE OF WILLIAM & MARY,Public Higher Education,228 Jamestown Road,NaN,NaN,NaN,Williamsburg,VA,Virginia,23185,US,United States of America,US,United States of America,COLIN SMOLINSKY,+1 757-221-2633,2022-04-06T16:07:48.219972-04:00
1,187711,HEALTH INDUSTRY DISTRIBUTORS ASSOCIATION,https://lda.senate.gov/api/v1/filings/d59dc0de...,d59dc0de-e9dd-42f0-b257-54a6f4085640,Q1,1st Quarter - Report,2013,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/d...,text/html,NaN,30522.92,a,Method A - Reporting amounts using LDA definit...,"Aaron Frazier, Senior Government Affairs Manager",2013-01-18T16:22:20.043000-05:00,NaN,United States of America,NaN,310 Montgomery Street,NaN,False,Alexandria,VA,22314.0,"{'id': 40011087, 'url': 'https://lda.senate.go...","{'id': 187711, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'HCR', 'general_issue_...",[],[],[],['HCR'],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,https://lda.senate.gov/api/v1/clients/187711/,75,NaN,False,True,VA,Virginia,US,United States of America,VA,Virginia,US,United States of America,2008-01-30,40011087,https://lda.senate.gov/api/v1/registrants/4001...,39783.0,HEALTH INDUSTRY DISTRIBUTORS ASSOCIATION,Government Affairs Analyst,510 King Street,Suite 200,NaN,NaN,Alexandria,VA,Virginia,22314,

In [53]:
# Cleaning some useless lobbying data and sort the c_id in an increasing order.
lda2013_Q1= lda2013_Q1.loc[:,['c_id','c_name', 'filing_year', 'filing_type', 'income', 'expenses']]
lda2013_Q1.sort_values(by='c_id',inplace=True)
lda2013_Q1.head(5)

,c_id,c_name,filing_year,filing_type,income,expenses
82,100012,LOCKHEED MARTIN,2013,Q1,10000.0,NaN
13194,100016,"ABB, INC.",2013,Q1,NaN,NaN
6390,100020,AMERICAN BANKERS ASSOCIATION SECURITIES ASSOCI...,2013,Q1,NaN,20000.0
12038,100021,ABBOTT LABORATORIES,2013,Q1,NaN,500000.0
13477,100023,ACADEMY FOR RADIOLOGY & BIOMEDICAL IMAGING RES...,2013,Q1,NaN,40000.0


In [56]:
company.head(5)

,ticker,title,assets,sector,industry,assets_summary
0,MSFT,MICROSOFT CORP,"{'label': 'Assets', 'description': 'Sum of the...",Technology,Software—Infrastructure,end val quarter year 0 ...
1,AAPL,Apple Inc.,"{'label': 'Assets', 'description': 'Sum of the...",Technology,Consumer Electronics,end val quarter year 0 ...
2,NVDA,NVIDIA CORP,"{'label': 'Assets', 'description': 'Sum of the...",Technology,Semiconductors,end val quarter year 0 ...
3,AMZN,AMAZON COM INC,"{'label': 'Assets', 'description': 'Sum of the...",Consumer Cyclical,Internet Retail,end val quarter year 0 ...
4,GOOGL,Alphabet Inc.,"{'label': 'Assets', 'description': 'Sum of the...",Communication Services,Internet Content & Information,end val quarter year 0 ...


In [64]:
# Remove duplicate company names in the company data
unique_name = company.drop_duplicates(subset='title')
unique_name = unique_name[['ticker','title']]
unique_name.info

<bound method DataFrame.info of      ticker                                 title
0      MSFT                        MICROSOFT CORP
1      AAPL                            Apple Inc.
2      NVDA                           NVIDIA CORP
3      AMZN                        AMAZON COM INC
4     GOOGL                         Alphabet Inc.
...     ...                                   ...
2627    DJT  Trump Media & Technology Group Corp.
2628   LENZ               LENZ Therapeutics, Inc.
2843  FRLAW         Fortune Rise Acquisition Corp
3042  VSACW      VISION SENSING ACQUISITION CORP.
3063  FHLTU               Future Health ESG Corp.

[2586 rows x 2 columns]>

In [76]:
# Reindexing
n = len(unique_name)
unique_name.set_index(pd.Series(range(0,n)),inplace=True)

In [62]:
# Remove duplicate company names in the lobbying data
unique_lda_name = lda2013_Q1.drop_duplicates(subset='c_name')
unique_lda_name = lda2013_Q1[['c_id','c_name']]
unique_lda_name

,c_id,c_name
82,100012,LOCKHEED MARTIN
13194,100016,"ABB, INC."
6390,100020,AMERICAN BANKERS ASSOCIATION SECURITIES ASSOCI...
12038,100021,ABBOTT LABORATORIES
13477,100023,ACADEMY FOR RADIOLOGY & BIOMEDICAL IMAGING RES...
...,...,...
14030,198076,CASH AMERICA INTERNATIONAL
14039,198092,"CITIZEN GUARDIAN, INC."
14066,198546,NATIONAL DOMESTIC WORKERS ALLIANCE
14094,198897,NATIONAL ASSOCIATION OF STATE TREASURERS


In [82]:
# Define a threshold for similarity
threshold = 80 

similar_names_dict = {}

# Iterate over titles in the first DataFrame
for title in unique_name['title']:
    # Find similar names in the second DataFrame using fuzz ratio
    similar_names = []
    for index, row in unique_lda_name.iterrows():
        if fuzz.ratio(title, row['c_name']) > 80:  # Adjust threshold as needed
            similar_names.append((row['c_id'], row['c_name']))
    similar_names_dict[title] = similar_names

KeyboardInterrupt: 